# Overview

This notebook provides a core set of analysis on a policy, primary focused on least-privilege and system self-protection. This is geared towards systems that have a customized SELinux policy based upon reference policy (and usually a RHEL or Fedora derivative).

To use this notebook you should:

   * Replace the policies loaded with the system policies that you are interested in.
   * Run each analysis and examine the output - this notebook doesn't give you yes/no answers. It requires interpretation.
   
# Getting the Policies

To use these notebook you need the source and binary versions of two policies: the system and base policy. Base policy, here, meaning the policy that the system developers started with. This is typically the policy from the version of RHEL or Fedora that the system is based upon.

System developers typically either add policy modules, directly modify the base policy, or both. Regardless of the approach, it's nice to have the base policy in both source and binary form.

Getting the source and binary policies for the system is dependent on how the system is developed.

The easiest way to get the source for the base policy is to start with the source for selinux-policy. You can grab those from:

   * http://vault.centos.org/
   * http://archives.fedoraproject.org/pub/archive/fedora/linux/updates/
   
Once you have the source rpm, you need to at least partially build it so that all of the patches are applied. First setup your system to build rpms - there is some info at https://wiki.centos.org/HowTos/RebuildSRPM. From there, just rpmbuild -bp will output the patched source to ~/rpmbuild/BUILD.

You can also build the binary policy this way, but it's actually kind of pain. It's easier to just grab the rpm and extract it with:

`rpm2cpio selinux-policy-targeted-3.13.1-190.fc24.noarch.rpm | cpio -idmv`

After you have done this, the policy should be under etc/selinux/targeted/policy/policy.30 (or similar) in the current directory.

Once you've collected the policies, simply save the source and binaries under the same directory with this notebook.

## Import and load the policies

You should change policy_paths.config to match the policies that you want to analyze.

As an example for this notebook, we are going to use a Fedora 25 policy as the system policy and a Fedora 24 policy as the base policy.

In [ ]:
# Import senotebook - this complication is just to handle running this in the development tree
try:
    import span as se
except:
    import os
    path = os.path.dirname(os.getcwd())
    import sys
    sys.path.insert(0, path)
    import span as se

import pandas as pd
from IPython.display import display, HTML

# P - binary system policy
# ps - source system policy
# bp - binary base policy
# bps - source base policy
p, ps, bp, bps = se.load_policies_from_config("policy_paths.config")

# Basic Policy Summary

## Numer of Types

In [ ]:
p_types = set(se.as_str(p.types()))
print("total types in custom policy: %d" % len(p_types))

p_domains = p.domain_types()
print("total number of domain types: %d" % len(p_domains))

## Custom Types

This is a quick way to figure out the _potentially_ custom types. It's only potentially, because it's possible that the base policy may have backported policy modules from newer policies and included them. So types not in the base policy would be present but not really custom to the system being analyzed.

Having said that, this approach is typically a good first approximation.

In [ ]:
custom_types, custom_domains = p.new_types(bp)

print("total number of potentially custom types: %d" % len(custom_types))
print("total number of potentially custom domains: %d" % len(custom_domains))

In [ ]:
p.types_summary(custom_domains)

In [ ]:
p.types_summary(custom_types)

## Initial SIDs
This is the list of initial security identifiers for object classes present in this policy.

In [ ]:
def initsids_as_dataframe(p):
    initsids = [{ "name": str(x), "context": str(x.context)} for x in p.initialsids()]
    return pd.DataFrame(initsids)[["name", "context"]]
    
initsids_as_dataframe(p)

In [ ]:
p_initsids = {str(x): str(x.context) for x in p.initialsids()}
b_initsids = {str(x): str(x.context) for x in bp.initialsids()}

Any new or missing initial sids?

In [ ]:
new_initsids = set(p_initsids.keys()) - set(b_initsids.keys())
print("new initial sids: " + str(new_initsids))

missing_initsids = set(b_initsids.keys()) - set(p_initsids.keys())
print("missing initial sids: " + str(missing_initsids))

Changed labels?

In [ ]:
changed_initsids = []
for i in p_initsids.keys():
    if not i in b_initsids:
        changed_initsids.append({"name": i, "system": p_initsids[i], "base": "NOT PRESENT"})
        continue
    if p_initsids[i] != b_initsids[i]:
        changed_initsids.append({"name": i, "system": p_initsids[i], "base": b_initsids[i]})
        
if len(changed_initsids) > 0:
    display(pd.DataFrame(changed_initsids)[["name", "system", "base"]])
else:
    print("No changed initial sid labels")


## Object Classes
This is the list of new object classes, both kernel and userspace, present in the policy.

In [ ]:
se.as_strset(p.classes()) - se.as_strset(bp.classes())

New permissions or missing permissions:

In [ ]:
# We are going to do commons separately becase they can be named the same as
# object classes
p_commons = {str(x): se.as_strset(x.perms) for x in p.commons()}
p_classes = {str(x): se.as_strset(x.perms) for x in p.classes()}
b_commons = {str(x): se.as_strset(x.perms) for x in bp.commons()}
b_classes = {str(x): se.as_strset(x.perms) for x in bp.classes()}

print("new commons: " + str(set(p_commons.keys()) - set(b_commons.keys())))
print("new object classes: " + str(set(p_classes.keys()) - set(b_classes.keys())))
print("removed object classes: " + str(set(b_classes.keys()) - set(p_classes.keys())))
print("removed commons: " + str(set(b_commons.keys()) - set(p_commons.keys())))


Changed permissions:

In [ ]:
def find_object_changes(name, pc, bc):
    cc = []
    for c in pc:
        if not c in bc:
            cc.append({"name": c, "added": str(pc[c]), "removed": "New Class"})
            continue
        added = pc[c] - bc[c]
        removed = bc[c] - pc[c]

        if len(added) or len(removed):
            cc.append({"name": c, "added": str(added), "removed": str(removed)})

    if len(cc) > 0:
        print("Changed %s permissions:" % name)
        display(pd.DataFrame(cc)[["name", "added", "removed"]])
    else:
        print("No changed %s permissions." % name)
        
find_object_changes("common", p_commons, b_commons)
find_object_changes("object class", p_classes, b_classes)
    

## Constraints
Since analysis tools do not address constraints in a meaningful way, here is a raw dump of the constraints for reference.

In [ ]:
HTML(ps.diff_mls_constraints(bps))

In [ ]:
HTML(ps.diff_mcs_constraints(bps))

In [ ]:
HTML(ps.diff_constraints(bps))

## Policy Caps

Policy capabilities control how policy enforcement is done on a system and is used to control optional features and backwards compatibility. Some documentation can be found at https://selinuxproject.org/page/Policy_Configuration_Statements.

In [ ]:
se.as_str(p.polcaps())

Changed policy capabilities:

In [ ]:
se.as_strset(p.polcaps()) ^ se.as_strset(bp.polcaps())

# System Self Protection Analysis

Analyses of access that can compromise the ability of the system to protect itself from malicious processes.

## Write Rules - boot_t
Files labeled boot_t include files for bootstrapping the system, including the critical kernel image and initial ramdisk image. Domains that can modify these files can completely compromise a system by replacing the kernel or initrd.

In [ ]:
print(ps.file_contexts("boot_t"))

In [ ]:
p.object_info_flow(object_type="boot_t", direction="w")

## Read Rules - system_map_t
Files labeled system_map_t contain the kernel symbol table which can be used to lookup the address of symbols in the kernel to aid in exploitation of the system. 

In [ ]:
print(ps.file_contexts("system_map_t"))

In [ ]:
p.object_info_flow(object_type="system_map_t", direction="r")

## Write Rules - modules_object_t
modules_object_t is the label placed on kernel modules. Files with this label contain kernel code that, depending on other aspects of system configuration, can be automatically loaded into the kernel as needed. Domains that can write to these files can compromise the integrity of the system.

In [ ]:
p.object_info_flow(object_type="modules_object_t", direction="w", tclass=["file", "dir"])

## Read/Write Rules - memory_device_t
Files labeled memory_device_t provide access to the raw physical memory of the solution. As such, reading from these files can compromise the confidentiality of the solution. Writing to these files can compromise the integrity of the solution.

In [ ]:
p.terules_query(target="memory_device_t", perms=["read"], tclass=["chr_file"])

In [ ]:
p.terules_query(target="memory_device_t", perms=["write", "append"], tclass=["chr_file"])

## Read/Write Rules - proc_kcore_t
Similar to files labeled proc_kcore_t provide access to the virtual memory of the solution. As such, reading from these files can compromise the confidentiality of the solution. Writing to these files can compromise the integrity of the solution.

In [ ]:
print(ps.genfscon("proc"))

In [ ]:
p.terules_query(target="proc_kcore_t", perms=["read"], tclass=["file"])

In [ ]:
p.terules_query(target="proc_kcore_t", perms=["write", "append"], tclass=["file"])

## Read/Write Rules - Terminal Types
Terminal files represent a large information flow between domains that can read/write from these possibly shared devices. One example - a domain writes a password prompt to another terminal and then reads the data entered by a user.

### Read Terminals

In [ ]:
p.terules_query(target=["console_device_t", "tty_device_t", "ptmx_t", "devpts_t"], perms=["read"], tclass=["chr_file"])

### Write Terminals

In [ ]:
p.terules_query(target=["console_device_t", "tty_device_t", "ptmx_t", "devpts_t"], perms=["write", "append", "ioctl"], tclass=["chr_file"])

## Write Rules - etc_t
Files labeled etc_t includes a variety of files for system-wide configuration. In addition, default poilicies often fail to split out sensitive files into separate types outside of etc_t. One example that can impact stateless systems is rwtab and statetab. Modifications to these files can alter what state is retained across reboots.

On solutions prior to RHEL 7, the default label for the password file /etc/passwd us also etc_t, thus any domain that can write to etc_t can modify the file containing key userdata (though no longer password hashes), e.g., changing an unpriv user ID to UID 0.

In [ ]:
p.object_info_flow(object_type="etc_t", tclass=["file", "dir"], direction="w")

## Write Rules - system_conf_t
Files labeled sys_conf_t are applied to system-wide configuration files, such as those residing in /etc/sysconfig. Included in that set, are the iptables configuration files. Domains with the ability to write to these files can alter the firewall ruleset, including SELinux packet labels applied via SECMARK, in effect, providing the ability to alter the intent of the policy.

In [ ]:
p.terules_query(target="system_conf_t", perms=["write", "append", "unlink"], tclass=["file"])

## Write Rules - net_conf_t
Files labeled net_conf_t are applied to network configuration files. These files contain the information the system uses to config network devices. Domains that can modify these files can alter the configuration of devices such  as eth0.

In [ ]:
p.object_info_flow(object_type="net_conf_t", direction="w")

## Write Rules - lib_t
Files labeled lib_t are used by dynamically linked applications. A vast number of shared libraries are labeled lib_t. Domains that can modify these files can alter the behavior of any application linked against these files.

In [ ]:
p.object_info_flow(object_type="lib_t", direction="w")

## Read/Write Rules - auditd_log_t var_log_t
Files labeled auditd_log_t contain the audit messages generated by the audit frameworks which is critical to monitoring the security posture of the solution. Files labeled var_log_t are typically files that should have a different label as var_log_t is a generic label and many base system domains have the ability to modify files with that label. Modifying these files can allow a domain to report mis-information or truncate otherwise sensitive log and audit entries.

In [ ]:
# Write Audit logs
p.object_info_flow(object_type="auditd_log_t", direction="w")

In [ ]:
# Read object logs
p.object_info_flow(object_type="auditd_log_t", direction="r")

In [ ]:
# Write generic logs
p.object_info_flow(object_type="var_log_t", direction="w")

## Read/Write Rules - shadow_t
Files labeled shadow_t contain user password data, including salts and hashes. Domains that can write to these files can modify user passwords.

Domains that can read from these files can collect user password data for use in, e.g., brute force attacks.

In [ ]:
p.object_info_flow(object_type="shadow_t", tclass=["file"], direction="w")

In [ ]:
p.object_info_flow(object_type="shadow_t", tclass=["file"], direction="r")

## Read/Write Rules - unlabeled_t Files
unlabeled_t is the default label for a number of object classes, such as packets and files. Almost nothing should have access to this type.

In [ ]:
p.object_info_flow(object_type="unlabeled_t", direction="w")

In [ ]:
p.object_info_flow(object_type="unlabeled_t", direction="r")

## Write Rules - wtmp_t
wtmp contains login data - another component of overall system logging and auditing. Writing to this allows one to modify the login records of the system.

In [ ]:
p.object_info_flow(object_type="wtmp_t", direction="w")

## Write Rules - shell_exec_t
Shell binaries are relied upon by numerous system components. The ability to modify these binaries represents significant risk.

In [ ]:
p.object_info_flow(object_type="shell_exec_t", direction="w")

## Write Rules - bin_t
Binaries labeled bin_t are relied upon by numerous system components. The ability to modify these binaries represents significant risk.

In [ ]:
p.object_info_flow(object_type="bin_t", direction="w")

## Read/Write Rules - fixed_disk_device_t
Fixed storage devices e.g., HDDs, should have strictly controlled raw write access, or write access in general. Writing to fixed disk devicces can compromise the integrity of the system. Reading from fixed disk devices can compromise the confidentiality of data residing on the system.

In [ ]:
# Write to fixed disk devices
p.terules_query(target="fixed_disk_device_t", perms=["write"], tclass=["blk_file"])

In [ ]:
# Read from fixed disk devices
p.terules_query(target="fixed_disk_device_t", perms=["read"], tclass=["blk_file"])

## Write Rules - default_t
Anything labeled default_t is in indication a more-specific label does not exist, which means there is a problem with the policy.

In [ ]:
p.object_info_flow(object_type="default_t", direction="w")

## Transition - initrc_t
On versions <= RHEL 6, initrc_t is a very powerful domain. Domains that can transition to initrc_t can start, stop, or otherwise influence services running on the system.

In [ ]:
p.domains_with(target_name="initrc_t", tclass=["process"], perms=["transition"])

## Memory Permissions
These permissions control access to specific operations on memory segments. Granting these permissions makes it easier to exploit a vulnerable process and should be avoided. Most can be addressed via code or compilation flag changes.

In [ ]:
p.domains_with(target_name="", tclass=["process"], perms=["execmem", "execmod", "execheap", "execstack"])

## SELinux

The ability to load policy, toggle enforcement, and change booleans is obviously important for system protection.

In [ ]:
p.domains_with(target_name="", tclass=["security"], perms=["load_policy", "setbool", "setenforce"], expand_attrs=True)